In [ ]:
# import the necessary packages
from imutils import contours, grab_contours
import numpy as np
import cv2
import ipywidgets as wd
import io
from PIL import Image
import base64

In [ ]:
# a function resizing to the smaller width and concatenating multiple images vertically  
def vconcat_resize(img_list, interpolation = cv2.INTER_CUBIC):
    
    # take minimum width 
    w_min = min(img.shape[1] for img in img_list) 
      
    # resizing images 
    im_list_resize = [
        cv2.resize(img,
                   (w_min, int(img.shape[0] * w_min / img.shape[1])),
                   interpolation = interpolation
                  ) 
        for img in img_list
    ] 
    # return final image 
    return cv2.vconcat(im_list_resize) 

In [ ]:
# ordering coordinates in top-left, top-right, bottom-right, and bottom-left order
def order_points(pts):
    # sort the points based on their x-coordinates
    xSorted = pts[np.argsort(pts[:, 0]), :]

    # grab the left-most and right-most points from the sorted
    # x-roodinate points
    leftMost = xSorted[:2, :]
    rightMost = xSorted[2:, :]

    # now, sort the left-most coordinates according to their
    # y-coordinates so we can grab the top-left and bottom-left
    # points, respectively
    leftMost = leftMost[np.argsort(leftMost[:, 1]), :]
    (tl, bl) = leftMost

    # now that we have the top-left coordinate, use it as an
    # anchor to calculate the Euclidean distance between the
    # top-left and right-most points; by the Pythagorean
    # theorem, the point with the largest distance will be
    # our bottom-right point
    #D = dist.cdist(tl[np.newaxis], rightMost, "euclidean")[0]
    D = [np.linalg.norm(tl - x) for x in rightMost]
    (br, tr) = rightMost[np.argsort(D)[::-1], :]

    # return the coordinates in top-left, top-right,
    # bottom-right, and bottom-left order
    return np.array([tl, tr, br, bl], dtype="float32")

In [ ]:
def measure_reference_object(ref, ref_length, dim = 'width'):
    
    # compute the rotated bounding box of the reference contour
    box = cv2.minAreaRect(ref)
    box = cv2.boxPoints(box)
    box = np.array(box, dtype="int")

    # order the points so that they appear in top-left, top-right,
    # bottom-right, and bottom-left order
    box = order_points(box)

    # unpack the ordered bounding box, then compute heigth and width
    (tl, tr, br, bl) = box
    height = np.linalg.norm(tl - bl)
    width = np.linalg.norm(tl - tr)


    # compute pixels per metric
    if dim == 'width':
        return width / ref_length
    elif dim == 'height':
        return height / ref_length
    else:
        print('Invalid dimension flag. Default to width')
        return width / ref_length

In [ ]:
def compute_contours(image):
    
    # convert image to grayscale, and blur it slightly
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    gray = cv2.GaussianBlur(gray, (7, 7), 0)
    
    # perform edge detection, then perform a dilation and erosion to
    # close gaps in between object edges
    edged = cv2.Canny(gray, 50, 100)
    edged = cv2.dilate(edged, None, iterations=1)
    edged = cv2.erode(edged, None, iterations=1)

    # find contours in the edge map
    cnts = cv2.findContours(edged.copy(), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    cnts = grab_contours(cnts)
    
    # sort contours
    (cnts, _) = contours.sort_contours(cnts, method="top-to-bottom")

    return cnts

In [ ]:
def on_run(data):
    [uploader, width, minimum_inclusion_size, visualisation] = data

    images = []
    results = ''
    
    # loop over the uploaded imags
    for name, file_info in uploader.value.items():
        
        # conert into an opencv format
        img = Image.open(io.BytesIO(file_info['content'])).convert('RGB')
        open_cv_image = np.array(img) 
        image = open_cv_image[:, :, ::-1].copy() # Convert RGB to BGR 

        # process the image and find contours
        cnts = compute_contours(image)

        # calculate the 'pixels per metric' calibration variable
        pixelsPerMetric = measure_reference_object(cnts[0], width)

        # drop the reference object
        #cnts = cnts[1:] - at the moment the reference object is recognised as 2 object so the first 2 are dropped
        cnts = cnts[2:]

        # preserve results from previous images
        results = results + name

        
        if visualisation == True:
            
            # loop over the contours individually
            for c in cnts:

                # compute convex hull of the contour
                c = cv2.convexHull(c)
                area = cv2.contourArea(c) / np.power(pixelsPerMetric, 2)

                # if the contour is not sufficiently large, ignore it
                if area <  minimum_inclusion_size:
                    continue

                # draw the contours
                cv2.drawContours(image, c, -1, (0, 0, 255), 5)

                # add area label underneath the contours 
                bottommost = list(c[c[:,:,1].argmax()][0])
                bottommost = (bottommost[0]-20, bottommost[1]+20)
                cv2.putText(image, "{:.1f}microns2".format(area), bottommost, cv2.FONT_HERSHEY_SIMPLEX,
                            0.65, (255, 255, 255), 2)

                # save results in a csv format
                results = results +',' + str(area) + '\n' 


            #  print name of the image on its top
            cv2.putText(image, name, (int(image.shape[1]/3), 30),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.65, (255, 255, 255), 2)

            # store the modified image
            images.append(image)
            
            
        else:
            # loop over the contours individually
            for c in cnts:

                # compute convex hull of the contour
                c = cv2.convexHull(c)
                area = cv2.contourArea(c) / np.power(pixelsPerMetric, 2)

                # if the contour is not sufficiently large, ignore it
                if area <  minimum_inclusion_size:
                    continue

                # store results
                results = results +',' + str(area) + '\n' 
        

    
    if visualisation == True:
        if len(images) > 1:
            image = vconcat_resize(images)

        is_success, im_buf_arr = cv2.imencode(".png", image)
        byte_im = im_buf_arr.tobytes()    

        # show the output image
        output_image.value = byte_im
        output_image.layout.visibility = 'visible'
   
    # create a link with results
    link_results(results)

In [ ]:
def link_results(results):
    
   # create the results file
   filename = 'chlamydia_inclusion_size.csv'
   b64 = base64.b64encode(results.encode())
   payload = b64.decode()

   # create an HTML button
   html_buttons = '''<html>

   <head>
   <meta name="viewport" content="width=device-width, initial-scale=1">
   </head>
   <body>
   <a download="{filename}" href="data:text/csv;base64,{payload}" download>
   <button class="p-Widget jupyter-widgets jupyter-button widget-button mod-warning">Download File</button>
   </a>
   </body>
   </html>
   '''
    
   # format and display the button
   html_button = html_buttons.format(payload=payload,filename=filename)
   display(wd.HTML(html_button))

In [ ]:
# create widgets
# upload button
uploader = wd.FileUpload(
    accept='.jpg, .png',
    multiple=True
)

# reference width entry box
width = wd.FloatText(
    value=10.0,
    description = "Width:",
    layout=wd.Layout(width='200px'),
    style = {'description_width': 'initial'}
)

# minimum inclusion size entry box
minimum_inclusion_size = wd.FloatText(
    value=20.0,
    description = "Minimum inclusion size:",
    style = {'description_width': 'initial'},
    layout=wd.Layout(width='200px'),
)

# visualisation checkbox
visualisation = wd.Checkbox(
    value=True,
    description='Visualisation',
    indent = False
)

# "Run button"
button = wd.Button(
    description='Measure chlamydia!',
    tooltip='Click me'
)
button.style.button_color = 'pink'

# image display widget
output_image = wd.Image(
    value=b'',
    width=500
)
# initially hidden
output_image.layout.visibility = 'hidden'

# output widget
out = wd.Output()

# define what happens on clicking the 'Measure chlamydia!' button
# let out capture the output
@out.capture()
def on_button_clicked(b):
    out.clear_output()
    output_image.layout.visibility = 'hidden'
    on_run([uploader,
            width.value,
            minimum_inclusion_size.value,
            visualisation.value]
          )
    
# attach a function to the 'Measure chlamydia!' button
button.on_click(on_button_clicked)

# display all widgets
display(wd.VBox([uploader,
                 visualisation,
                 width,
                 minimum_inclusion_size,
                 button,
                 out,
                 output_image]
               )
       )